In [9]:
from ultralytics import YOLO
from collections import Counter
import cv2

In [10]:
import json

with open("../nutrition/class_id_map.json") as f:
    CLASS_ID_MAP = json.load(f)

In [ ]:
model = YOLO("../runs/yolov11_fnv_correct_p1/weights/best.pt") # change "yolov11_fnv_correct_p1" to your file name
print(model.names)

{0: '0', 1: '1', 2: '10', 3: '11', 4: '12', 5: '13', 6: '14', 7: '15', 8: '16', 9: '17', 10: '18', 11: '19', 12: '2', 13: '20', 14: '21', 15: '22', 16: '23', 17: '24', 18: '25', 19: '26', 20: '27', 21: '28', 22: '29', 23: '3', 24: '30', 25: '31', 26: '32', 27: '33', 28: '34', 29: '35', 30: '36', 31: '37', 32: '38', 33: '39', 34: '4', 35: '40', 36: '5', 37: '6', 38: '7', 39: '8', 40: '9'}


In [12]:
def run_detection(image, conf_thresh=0.4):
    results = model(image, conf=conf_thresh)[0]
    
    detections = []
    for box in results.boxes:
        cls_id = int(box.cls)
        detections.append({
            "class_id": cls_id,
            "class_name": CLASS_ID_MAP[str(cls_id)],
            "confidence": float(box.conf),
            "bbox": box.xyxy.tolist()[0]
        })
    return detections

In [13]:
img = cv2.imread("../assets/example_images/sample.jpg")
detections = run_detection(img)

detections


0: 640x640 5 36s, 3 7s, 6.7ms
Speed: 8.3ms preprocess, 6.7ms inference, 5.5ms postprocess per image at shape (1, 3, 640, 640)


[{'class_id': 30,
  'class_name': 'strawberry',
  'confidence': 0.8107065558433533,
  'bbox': [366.8553466796875,
   1046.482421875,
   721.0140991210938,
   1363.8590087890625]},
 {'class_id': 38,
  'class_name': 'broccoli',
  'confidence': 0.7432713508605957,
  'bbox': [1195.0335693359375,
   1332.627197265625,
   1399.885986328125,
   1556.129638671875]},
 {'class_id': 30,
  'class_name': 'strawberry',
  'confidence': 0.7307214736938477,
  'bbox': [387.0347595214844,
   754.2926635742188,
   602.7429809570312,
   990.5155639648438]},
 {'class_id': 30,
  'class_name': 'strawberry',
  'confidence': 0.6218421459197998,
  'bbox': [169.56967163085938,
   769.1058959960938,
   398.690673828125,
   975.5800170898438]},
 {'class_id': 38,
  'class_name': 'broccoli',
  'confidence': 0.5576357245445251,
  'bbox': [1449.51513671875,
   995.5081787109375,
   1850.0887451171875,
   1261.8048095703125]},
 {'class_id': 30,
  'class_name': 'strawberry',
  'confidence': 0.5575394630432129,
  'bbox': 

In [14]:
def count_items(detections, conf_thresh=0.4):
    filtered = [
        d["class_name"] for d in detections
        if d["confidence"] >= conf_thresh
    ]
    return dict(Counter(filtered))

In [15]:
count_items(detections)

{'strawberry': 5, 'broccoli': 3}